In [1]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
#from keybert import KeyBERT
import yake
import string

In [2]:
sds_text = pd.read_csv('../data/sds_text.csv')

<IPython.core.display.Javascript object>

In [3]:
sds_text

,episode_name,length_episode,context_episode,guest_name,guest_info,text_episode,episode_number,episode_date,episode_year,episode_day,host_episode,episode_split_text,speaker
0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",2016,Saturday,Kirill Eremenko,This is episode number one with ex-chemical en...,Kirill
1,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",2016,Saturday,Kirill Eremenko,"Hey guys, welcome to the Podcast. I’ve got Rub...",Kirill
2,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",2016,Saturday,Kirill Eremenko,Thank you! Thanks for having me over. I’m doin...,Ruben
3,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",2016,Saturday,Kirill Eremenko,Awesome. It’s great to hear you and for those ...,Kirill
4,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",2016,Saturday,Kirill Eremenko,"Yeah sure. So, I’m the senior manager of analy...",Ruben
...,...,...,...,...,...,...,...,...,...,...,...,...,...
52607,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,"May 30, 2023",2023,Tuesday,Jon Krohn,"Yeah, right. Yeah, as I mean, it actually, it ...",Jon Krohn
52608,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,"May 30, 2023",2023,Tuesday,Jon Krohn,Happy to.,Matar Haller
52609,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,"May 30, 2023",2023,Tuesday,Jon Krohn,"Nice. Well, yeah, so you mentioned potentially...",Jon Krohn
52610,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,"May 30, 2023",2023,Tuesday,Jon Krohn,Thank you for having me. This was fascinating ...,Matar Haller


In [4]:
sds_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52612 entries, 0 to 52611
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   episode_name        52612 non-null  object
 1   length_episode      52612 non-null  int64 
 2   context_episode     52612 non-null  object
 3   guest_name          52612 non-null  object
 4   guest_info          52612 non-null  object
 5   text_episode        52610 non-null  object
 6   episode_number      52612 non-null  int64 
 7   episode_date        52612 non-null  object
 8   episode_year        52612 non-null  int64 
 9   episode_day         52612 non-null  object
 10  host_episode        52612 non-null  object
 11  episode_split_text  52610 non-null  object
 12  speaker             52235 non-null  object
dtypes: int64(3), object(10)
memory usage: 5.2+ MB


In [5]:
sds_processed = pd.DataFrame(sds_text.groupby(['episode_number', 'episode_name', 'length_episode', 'context_episode', 'guest_name', 'host_episode', 'episode_year'])['episode_split_text'].agg(lambda x: ' '.join(str(i) for i in x))).reset_index()

<IPython.core.display.Javascript object>

In [6]:
sds_processed

,episode_number,episode_name,length_episode,context_episode,guest_name,host_episode,episode_year,episode_split_text
0,1,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,Kirill Eremenko,2016,This is episode number one with ex-chemical en...
1,2,"Machine Learning, Recommender Systems and the...",51,Machine Learning Data Science,Hadelin de Ponteves,Kirill Eremenko,2016,This is session number two with machine learni...
2,3,"Defining the Data Problem, Data Science in Ma...",53,Machine Learning R Programming Data Science,Dr. Wilson Pok,Kirill Eremenko,2016,This is episode number three with Nanophysics ...
3,4,"Data and Strategy, three Pillars of Research ...",60,Business Data Science,Brendan Hogan,Kirill Eremenko,2016,This is episode four with business strategy ex...
4,5,"Computer Forensics, Fraud Analytics and knowi...",63,Data Science,Dmitry Korneev,Kirill Eremenko,2016,This is episode number five with forensics inv...
...,...,...,...,...,...,...,...,...
678,679,"The A.I. and Machine Learning Landscape, with...",94,Business Data Science Artificial Intelligence,George Mathew,Jon Krohn,2023,This is episode number 679 with George Matthew...
679,680,Automating Industrial Machines with Data Scie...,30,Business Data Science,Allegra Alessi,Jon Krohn,2023,This is episode number 680 with Allegra Alessi...
680,681,XGBoost,72,Machine Learning Data Science Python,Matt Harrison,Jon Krohn,2023,"This is episode number 681 with Matt Harrison,..."
681,682,"Business Intelligence Tools, with Mico Yuk",28,Business Data Science,Mico Yuk,Jon Krohn,2023,"This is episode number 682 with Mico Yuk, host..."


In [7]:
sds_processed['episode_split_text'].loc[30]

"This is episode number 31, with my good friend David Tanaskovic.Welcome to the SuperDataScience podcast. My name is Kirill Eremenko, data science coach and lifestyle entrepreneur. And each week we bring you inspiring people and ideas to help you build your successful career in data science. Thanks for being here today and now let’s make the complex simple.Hello and welcome to the SuperDataScience podcast. Super excited to have you on board, and today I have invited my good friend David Tanaskovic onto the show. So something you need to know about David is that he's the happiest person I know on this planet. He's always super excited about life, nothing can bring him down, and he's just generally having fun going through life. Like you know sometimes you wish you would enjoy the things you do and your day to day activities, more like sometimes a day passes by and you feel like oh, wow, like I didn't really enjoy this day as much as I could've, or as much as I wish I had. Well I think t

In [8]:
# Function to clean the text 
def processed_text(text):  
    # Lowercase the text
    text = text.lower()
    # Remove punctuation from the text
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Remove digits from the text
    text = ''.join(char for char in text if not char.isdigit())
    return text

In [9]:
sds_processed['episode_split_text'] = sds_processed['episode_split_text'].apply(processed_text)

In [10]:
sds_processed

,episode_number,episode_name,length_episode,context_episode,guest_name,host_episode,episode_year,episode_split_text
0,1,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,Kirill Eremenko,2016,this is episode number one with exchemical eng...
1,2,"Machine Learning, Recommender Systems and the...",51,Machine Learning Data Science,Hadelin de Ponteves,Kirill Eremenko,2016,this is session number two with machine learni...
2,3,"Defining the Data Problem, Data Science in Ma...",53,Machine Learning R Programming Data Science,Dr. Wilson Pok,Kirill Eremenko,2016,this is episode number three with nanophysics ...
3,4,"Data and Strategy, three Pillars of Research ...",60,Business Data Science,Brendan Hogan,Kirill Eremenko,2016,this is episode four with business strategy ex...
4,5,"Computer Forensics, Fraud Analytics and knowi...",63,Data Science,Dmitry Korneev,Kirill Eremenko,2016,this is episode number five with forensics inv...
...,...,...,...,...,...,...,...,...
678,679,"The A.I. and Machine Learning Landscape, with...",94,Business Data Science Artificial Intelligence,George Mathew,Jon Krohn,2023,this is episode number with george matthew ma...
679,680,Automating Industrial Machines with Data Scie...,30,Business Data Science,Allegra Alessi,Jon Krohn,2023,this is episode number with allegra alessi io...
680,681,XGBoost,72,Machine Learning Data Science Python,Matt Harrison,Jon Krohn,2023,this is episode number with matt harrison man...
681,682,"Business Intelligence Tools, with Mico Yuk",28,Business Data Science,Mico Yuk,Jon Krohn,2023,this is episode number with mico yuk host of ...


In [11]:
sds_processed['episode_split_text'].loc[30]

'this is episode number  with my good friend david tanaskovicwelcome to the superdatascience podcast my name is kirill eremenko data science coach and lifestyle entrepreneur and each week we bring you inspiring people and ideas to help you build your successful career in data science thanks for being here today and now let’s make the complex simplehello and welcome to the superdatascience podcast super excited to have you on board and today i have invited my good friend david tanaskovic onto the show so something you need to know about david is that hes the happiest person i know on this planet hes always super excited about life nothing can bring him down and hes just generally having fun going through life like you know sometimes you wish you would enjoy the things you do and your day to day activities more like sometimes a day passes by and you feel like oh wow like i didnt really enjoy this day as much as i couldve or as much as i wish i had well i think that david never has that p

In [15]:
sds_processed['episode_keywords'] = None

In [16]:
# Function to generate 100 keywords per episode using Yake library to store keywords in one column as list
def yake_keyword_extractor(doc):
    global sds_processed
    
    for i, episode_text in doc.iteritems():
        max_ngram_size = 3
        deduplication_threshold = 0.3
        windowSize = 1
        numOfKeywords = 100
        kw_extractor = yake.KeywordExtractor(n = max_ngram_size, dedupLim = deduplication_threshold , windowsSize = windowSize, top = numOfKeywords)
        keywords = kw_extractor.extract_keywords(episode_text)
        sds_processed.at[i, 'episode_keywords'] = keywords

In [ ]:
yake_keyword_extractor(sds_processed['episode_split_text'])

In [ ]:
# Function to generate 100 keywords per episode using Yake library if keywords need to be separate columns

def yake_keyword_extractor(doc):
    global sds_processed

    for i, episode_text in doc.iteritems():
        max_ngram_size = 3
        deduplication_threshold = 0.3
        windowSize = 1
        numOfKeywords = 100
        kw_extractor = yake.KeywordExtractor(n=max_ngram_size, dedupLim=deduplication_threshold, windowsSize=windowSize, top=numOfKeywords)
        keywords = kw_extractor.extract_keywords(episode_text)

        # Create separate columns for each keyword
        for j, keyword in enumerate(keywords):
            column_name = f'episode_keyword{j+1}'
            sds_processed.at[i, column_name] = keyword[0]  # Store the keyword value in the respective column

In [ ]:
def tuple_extractor(row):
    words = [t[0].replace(" ", "_") for t in row]
    return " ".join(words)

In [ ]:
sds_processed['keywords'] = sds_processed['episode_keywords'].apply(tuple_extractor)

In [ ]:
sds_processed[sds_processed['episode_number'] == 31]

In [ ]:
def tuple_extractor(row):
    words = [t[0] for t in row]
    return words

In [ ]:
yake_keyword_extractor(sds_processed['episode_split_text'])

In [ ]:
sds_processed

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
 
    
document = ["One Geek helps Two Geeks",
            "Two Geeks help Four Geeks",
            "Each Geek helps many other Geeks at GeeksforGeeks"]

# Create a Vectorizer Object
vectorizer = CountVectorizer()
 
vectorizer.fit(document)
 
# Printing the identified Unique words along with their indices
print("Vocabulary: ", vectorizer.vocabulary_)
 
# Encode the Document
vector = vectorizer.transform(document)
 
# Summarizing the Encoded Texts
print("Encoded Document is:")
print(vector.toarray())

In [ ]:
sds_1.shape

In [ ]:
sds_processed

In [ ]:
sds_processed['episode_keywords'].loc[20]

In [ ]:
sds_processed.info()

In [ ]:
#sds_processed.to_csv('../data/sds_yake_keywords.csv')

In [ ]:
#sds_keywords = pd.read_csv('../data/sds_yake_keywords.csv')

In [ ]:
sds_1 = sds_processed[sds_processed['episode_number'] == 1]

In [ ]:
sds_1

In [ ]:
sds_1.info()

In [ ]:
sds_1['episode_keywords'].loc[0]

In [ ]:
sds_1

In [ ]:
sds_1['keywords'] = None

In [ ]:
sds_processed['keywords'] = None

In [ ]:
print(sds_processed['keywords'].loc[677])

In [ ]:
sds_1['episode_keywords'].apply(lambda x: tuple_extractor(x))

In [ ]:
sds_1.loc[0]

In [ ]:
sds_1['keywords'].loc[0]

In [ ]:
sds_1['keywords'].loc[0]

In [ ]:
def tuple_extractor(word_tuples):
    global sds_1
    words = [t[0] for t in word_tuples]
    return words

In [ ]:
sds_1['keywords'] = sds_1['episode_keywords'].apply(lambda x: tuple_extractor(x))

In [ ]:
sds_1

In [ ]:
sds_1['keywords'] = sds_1['episode_keywords'].apply(tuple_extractor)

In [ ]:
sds_1['keywords'].loc[0]

In [ ]:
def tuple_extractor(tuple):
    global sds_keywords
    words = []
    #for i, episode_text in doc.iteritems():
    for i, word_tuple in tuple.iteritems():
        words = [t[0] for t in word_tuple]
        sds_keywords['keywords'] = words
    
            

In [ ]:
sds_keywords['episode_keywords'].apply(tuple_extractor)

In [ ]:
sds_keywords

In [ ]:
sds_1['keywords'] = sds_1['episode_keywords'].apply(tuple_extractor)

In [ ]:
sds_1

In [ ]:
[t[0] for t in sds_1['episode_keywords']]

In [ ]:
sds_processed['episode_keywords'] = [t[0] for t in sds_processed['episode_keywords']]

In [ ]:
sds_processed

In [ ]:
sds_processed['episode_keywords'].loc[678]

In [ ]:
def keyword_extractor(doc):
    kw_model = KeyBERT()
    global sds_processed
    stopwords = list(STOP_WORDS)
    
    for i, episode_text in doc.iteritems():
        keywords = kw_model.extract_keywords(episode_text, keyphrase_ngram_range=(1, 1), stop_words = stopwords, top_n = 100, use_mmr=True, diversity=0.7)
        sds_processed.at[i, 'episode_keywords'] = keywords

In [ ]:
kw_model = KeyBERT()
stopwords = list(STOP_WORDS)
keywords = kw_model.extract_keywords(doca, keyphrase_ngram_range=(1, 2), stop_words = stopwords, top_n = 100, use_mmr=True, diversity=0.8)



In [ ]:
keywords

In [ ]:
keyword_extractor(sds_processed['episode_split_text'])

In [ ]:
len(keywords)

In [ ]:
keywords

In [ ]:
#sds_processed['episode_split_text'][1:5].apply(keyword_extractor)

In [ ]:
# https://maartengr.github.io/BERTopic/index.html